In [ ]:
import anndata as ad
import pandas as pd
import platform
import h5py

from anndata._io.specs import read_elem
from pathlib import Path

Current columns to my understanding to subset below can add more columns as we better learn the schema and what should be there/what is there

In [ ]:
PERTURB_COLUMNS = [
    'sample_name',
    'genetic_perturbation_strategy',
    'genetic_perturbation_id',
    'num_umis_guide_id',
]

The initial files are from /home/shared/home/jychien-40stanford-2eedu/lattice-tools/bcp/curated_matrices/

Sounds like this is still the most up to date files for 7.1

You can directly point to here in the hub, or i ended up adding them together in a tarred zip and then downloading locally and add into ~/Downloads/perturbed_h5ads or some other similar path

In [ ]:
hub_path = Path('/home/shared/home/jychien-40stanford-2eedu/lattice-tools/bcp/curated_matrices/')
local_path = Path.home() / 'Downloads/perturbed_h5ads'

perturbed_path =  hub_path if platform.machine() == "x86_64" else local_path

files = [file for file in perturbed_path.iterdir() if ".h5ad" in file.name]
files

Make list of dicts, loading only obs and uns from h5ads to see what all the data looks at in a more digestable format

In [ ]:
obs_uns = []
for path in files:
    with h5py.File(path) as f:
        obs_uns.append({
            'obs': read_elem(f['obs']),
            'uns': read_elem(f['uns']),
        })

obs_uns

In [ ]:
[f"File '{obs_uns_dict['uns']['title']}' obs shape: {obs_uns_dict['obs'].shape}" for obs_uns_dict in obs_uns]

In [ ]:
obs_df = pd.concat([meta['obs'] for meta in obs_uns])
obs_df

In [ ]:
obs_df.info()

In [ ]:
obs_df[PERTURB_COLUMNS]

Getting all the uns from the h5ads and then making into one large df to see basic pattenrs of the data

In [ ]:
uns_dicts = []
for meta in obs_uns:
    uns = meta['uns']
    title = uns['title']
    for genetic_perturbation_id, values in uns['genetic_perturbations'].items():
        values['genetic_perturbation_id'] = genetic_perturbation_id
        values['title'] = title
        uns_dicts.append(values)

uns_df = pd.DataFrame(uns_dicts)
uns_df